In [319]:
## Import necessary modules
import os,sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
import matplotlib.patches as mpatches

#sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/'))
#from ce_funclib import determine_kernzeit as dtkz

%matplotlib inline


## Import data frome pickle generated from muß ein file mit agentenstats sein
arcpth='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/'


In [320]:
######## GET A LIST OF MATCHING .xls FILES FROM THE GIVEN DIRECTORY

In [321]:
def collectxlfiles(arcpath):
    xlfilelist=list()

    for xlfile in os.listdir(arcpath):
        if xlfile.startswith('CE_al'):
            xlfileabs=os.path.join(arcpath,xlfile)
            xlfilelist.append(xlfileabs)
    return sorted(xlfilelist)

xlfilelist=collectxlfiles(arcpth)
#xlfilelist
#examplefile=xlfilelist[233]

In [322]:
def determine_kernzeit(datum):
    ### Kernzeiten
    ### ab 01.03.2017: Mo-Fr 11:30-19:30
    ### ab 05.06.2017: Mo-Fr 8-20
    ### ab 08.07. plus Samstag 8-13
    weekday=datum.strftime('%a')

    if datum.date() < datetime.date(2017,3,1):
        bzeit = 'k'

    elif datetime.date(2017,3,1) <= datum.date() < datetime.date(2017,6,5):   ## Zeit zwischen 1.Maerz und 1. Juni
        if weekday in ("Sat", "Sun"): ## WE immer Nebenzeit
            bzeit = 'n'
        else:                           ## Werktage von 11:30 bis 19:30
            if datetime.time(11,30) <= datum.time() < datetime.time(19,30):
                bzeit = 'k'
            else:
                bzeit = 'n'

    elif datetime.date(2017,6,5) <= datum.date() < datetime.date(2017,7,8): ## Zeit ab 05.Juni bis 07. Juli
        if weekday in ("Sat", "Sun"): ## WE immer Nebenzeit
            bzeit = 'n'
        else:                           ## Werktage von 8-20
            if datetime.time(8,00) <= datum.time() < datetime.time(20,00):
                bzeit = 'k'
            else:
                bzeit = 'n'
    ### hier noch ab wann samstags 8-13 gezaehlt wird
    elif datum.date() >= datetime.date(2017,7,8): ## Zeit ab Sa, 08. Juli
        if weekday in ("Sun"): ## So. Nebenzeit
            bzeit = 'n'   
        elif weekday in ("Sat"): ## Sa. 8-13
            if datetime.time(8,00) <= datum.time() < datetime.time(13,00):
                bzeit = 'k'
            else:
                bzeit = 'n'
        else:                           ## Werktage von 8-20
            if datetime.time(8,00) <= datum.time() < datetime.time(20,00):
                bzeit = 'k'
            else:
                bzeit = 'n'

    return bzeit

In [323]:
###### TEST FOR DATA IN FILE, SORT OUT EMPTY FILES 

In [324]:
def filetoframe(exfile):
    exframe=pd.read_excel(exfile) # this is a regular pd.DataFrame
    datecell=exframe.iloc[0,1]
    sheet_datetime=pd.to_datetime(datecell,format='%d.%m %Y : %H')
    sheet_date=sheet_datetime.date()
    
    integritycheck=exframe.iloc[2,1] # files with data have "agenten" here, files with no calls have a 'nan'

    if integritycheck != 'Agenten':
        # if it's empty, keep date for filling it later
        print('Exception: ', end='')
        except_status='ex'
        
        usefulcols={0:'tstamp',1:'agent',3:'an',4:'be',22:'vl',24:'ht_float',29:'tt_float'} # map cols to decent names
        exframe=exframe.reindex(columns=sorted(usefulcols.keys()))
        exframe.rename(columns=usefulcols,inplace=True)        
        exframe=exframe[0:1] # strip text rows and the mangled sum row
        print(sheet_datetime)
        exframe['tstamp']=sheet_datetime
        exframe['date']=sheet_date
        exframe['agent']='platzhalter'
        exframe.fillna(0, inplace=True)
        exframe[['wd','ww','mm','yy']]=exframe['tstamp'].dt.strftime('%a,%W,%m,%Y').str.split(',',expand=True) # make ww,yy,mm,wd columns
        exframe['bz']=exframe['tstamp'].apply(determine_kernzeit)
        exframe['ort']=exframe['agent'].str[0] # split the identifier into useable columns
        exframe['id']='foobar' # split the identifier into useable columns
        #exframe.set_index('tstamp', inplace=True)
        
        return exframe,except_status
        
    else:
        except_status='reg'
        
        exframe.columns=range(0,30) # rename columns to a temporarily more readable format, fancy rename later
        usefulcols={0:'tstamp',1:'agent',3:'an',4:'be',22:'vl',24:'ht_float',29:'tt_float'} # map cols to decent names
        exframe=exframe[sorted(usefulcols.keys())] # skip cols and keep the ones we need
        exframe.rename(columns=usefulcols,inplace=True) # rename cols
        exframe=exframe[3:-1] # strip text rows and the mangled sum row
        exframe['tstamp']=pd.to_datetime(exframe['tstamp'],format=' %d.%m.%Y %H:%M ')
        exframe['date']=exframe['tstamp'].dt.date
        exframe[['wd','ww','mm','yy']]=exframe['tstamp'].dt.strftime('%a,%W,%m,%Y').str.split(',',expand=True) # make ww,yy,mm,wd columns
        exframe['bz']=exframe['tstamp'].apply(determine_kernzeit)
        exframe['ort']=exframe['agent'].str[0] # split the identifier into useable columns
        exframe['id']=exframe['agent'].str[-6:] # split the identifier into useable columns
        exframe['agent']=exframe['agent'].str[2:-7] # split the identifier into useable columns

        
        return exframe,except_status



In [325]:
framelist=list()
exceptionlist=list()
for xfile in xlfilelist:
    
    #print('file:',xfile)
    frame_from_file,except_status=filetoframe(xfile)
    #print(frame_from_file.columns)
    #print(frame_from_file['date'])
    if except_status=='ex':
        exceptionlist.append(xfile)
    framelist.append(frame_from_file)

exceptionlist
    

Exception: 2017-04-17 00:00:00
Exception: 2017-05-14 00:00:00


['/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-04-17.xls',
 '/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-05-14.xls']

In [326]:
#### produce a unified frame with all data and sort it by timstamp and agentname
bigframeii=pd.concat(framelist)

bigframeii.sort_values(['tstamp','agent'],inplace=True)
bigframeii.reset_index(drop=True,inplace=True) # there you go



In [327]:
bigframeii

,tstamp,agent,an,be,vl,ht_float,tt_float,date,wd,ww,mm,yy,bz,ort,id
0,2017-03-04 08:00:00,beckfra,1,1,0,2.3667,2.1333,2017-03-04,Sat,09,03,2017,n,H,216694
1,2017-03-04 08:00:00,tetzlva,1,1,0,2.6833,2.6167,2017-03-04,Sat,09,03,2017,n,B,613887
2,2017-03-04 09:00:00,tetzlva,2,2,0,1.05,0.9167,2017-03-04,Sat,09,03,2017,n,B,613887
3,2017-03-04 10:00:00,scholan,1,1,0,2.1,0.6,2017-03-04,Sat,09,03,2017,n,H,189939
4,2017-03-04 11:00:00,beckfra,1,1,0,4.3333,4.2333,2017-03-04,Sat,09,03,2017,n,H,216694
5,2017-03-04 11:00:00,dehnsil,1,1,0,3.4,2.4667,2017-03-04,Sat,09,03,2017,n,H,386723
6,2017-03-04 12:00:00,tetzlva,3,3,0,6.5167,4.55,2017-03-04,Sat,09,03,2017,n,B,613887
7,2017-03-04 13:00:00,tetzlva,2,2,0,2.5167,2.4167,2017-03-04,Sat,09,03,2017,n,B,613887
8,2017-03-04 14:00:00,beckfra,1,1,0,2.6167,2.4833,2017-03-04,Sat,09,03,2017,n,H,216694
9,2017-03-04 15:00:00,gesinst,1,1,0,2.7833,2.1167,2017-03-04,Sat,09,03,2017,n,B,770628


In [336]:
### some date locator play
mai11_2017=datetime.date(2017,5,11)
mai=bigframeii.loc[bigframeii['mm'] == '05']
mai.iloc[0,0]
mai11=bigframeii.loc[bigframeii['date'] == mai11_2017]
mai11['ht_float'].sum()
print('everything correct')

everything correct
